In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io
import math
import matplotlib.gridspec as gridspec
# from plotting import newfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras import layers, activations
from scipy.interpolate import griddata
from eager_lbfgs import lbfgs, Struct
from pyDOE import lhs

C:\Users\milad\.conda\envs\tf\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\milad\.conda\envs\tf\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to ma

In [1]:
layer_sizes = [5, 250, 250, 250, 1]

sizes_w = []
sizes_b = []
for i, width in enumerate(layer_sizes):
    if i != 1:
        sizes_w.append(int(width * layer_sizes[1]))
        sizes_b.append(int(width if i != 0 else layer_sizes[1]))

def set_weights(model, w, sizes_w, sizes_b):
        for i, layer in enumerate(model.layers[0::2]):
            start_weights = sum(sizes_w[:i]) + sum(sizes_b[:i])
            end_weights = sum(sizes_w[:i+1]) + sum(sizes_b[:i])
            weights = w[start_weights:end_weights]
            w_div = int(sizes_w[i] / sizes_b[i])
            weights = tf.reshape(weights, [w_div, sizes_b[i]])
            biases = w[end_weights:end_weights + sizes_b[i]]
            weights_biases = [weights, biases]
            layer.set_weights(weights_biases)


def get_weights(model):
        w = []
        for layer in model.layers[0::2]:
            weights_biases = layer.get_weights()
            weights = weights_biases[0].flatten()
            biases = weights_biases[1]
            w.extend(weights)
            w.extend(biases)

        w = tf.convert_to_tensor(w)
        return w

# @tf.function
# def mish(x):
#     return x * tf.nn.tanh(tf.nn.softplus(x))
    
    
def neural_net(layer_sizes):
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(layer_sizes[0],)))
#     model.add(layers.LayerNormalization())
    for width in layer_sizes[1:-1]:
        model.add(layers.Dense(
            width, activation=tf.nn.silu,
            kernel_initializer="glorot_normal"))
        model.add(layers.Dropout(.5))
    model.add(layers.Dense(
            layer_sizes[-1], activation=None,
            kernel_initializer="glorot_normal"))
    return model


u_model = neural_net(layer_sizes)

u_model.summary()


@tf.function
def loss(x_f, t_f, d_f, kl_f, q_f,
         x0, t0, d0, kl0, q0,
         x_lb, t_lb, d_lb, kl_lb, q_lb,
         x_ub, t_ub, d_ub, kl_ub, q_ub,
         ub1_weights, ub2_weights, ui_weights):

#     u_pred = u_model(tf.concat([x_s_batch, t_s_batch, d_s_batch],1))
#     mse_s_u = tf.reduce_mean(tf.square(u_weights*(u_star - u_pred)))
    
    f_u_pred = f_model(x_f, t_f, d_f, kl_f, q_f)
#     , f_x_pred, f_t_pred
    u0_pred = u_model(tf.concat([x0, t0, d0, kl0, q0],1))
    u_lb_pred, u_lb_x_pred = u_x_model(x_lb, t_lb, d_lb, kl_lb, q_lb)
    u_ub_pred, u_ub_x_pred = u_x_model(x_ub, t_ub, d_ub, kl_ub, q_ub)

    mse_0_u = tf.reduce_mean(tf.square(ui_weights*u0_pred))
    
    flux_u_lb = 10*1 - 10*u_lb_pred + tf.math.multiply(d_lb, u_lb_x_pred)
    flux_u_ub = u_ub_x_pred
    #tf.tanh(20*t_lb)
    mse_b_u = tf.reduce_mean(tf.square(ub1_weights*flux_u_lb)) + tf.reduce_mean(tf.square(ub2_weights*flux_u_ub))

    mse_f_u = tf.reduce_mean(tf.square(f_u_pred))
#     mse_f_x = tf.reduce_mean(tf.square(f_x_pred))
#     mse_f_t = tf.reduce_mean(tf.square(f_t_pred))
#     mse_f_g = .1*mse_f_x + .1*mse_f_t


    return 1*mse_0_u + 1*mse_b_u + 1*mse_f_u, mse_0_u, mse_f_u, mse_b_u
# , mse_f_g
#  10*mse_s_u + 


@tf.function
def f_model(x, t, d, kl, q):
    #pe = tf.constant(3, dtype=tf.float32)
    u = u_model(tf.concat([x,t,d,kl,q], 1))
    u_x = tf.gradients(u,x)[0]
    u_xx = tf.gradients(u_x, x)[0]
    u_t = tf.gradients(u,t)[0]
#     Rf = 1 + 2*(tf.math.multiply(kl,q)/(1 + tf.math.multiply(kl,u))**2)
    f = u_t + 10*u_x - tf.math.multiply(d,u_xx) + (1.587/.37)*tf.gradients(tf.math.multiply(kl*q,u)/(1+tf.math.multiply(kl,u)),t)[0]
    
#     f_t1 = tf.gradients((1.587/.37)*tf.gradients(tf.math.multiply(kl*q,u)/(1+tf.math.multiply(kl,u)),t)[0],t)[0]
#     f_t2 = tf.gradients(u_t, t)[0]
#     f_t3 = tf.gradients(10*u_x, t)[0]
#     f_t4 = tf.gradients(tf.math.multiply(d,u_xx), t)[0]
#     f_t = f_t2 + f_t3 - f_t4 + f_t1
    
#     f_x = tf.gradients(f, x)[0]
#     f_t = tf.gradients(f, t)[0]
    
    return f


@tf.function
def u_x_model(x, t, d, kl, q):
    u = u_model(tf.concat([x,t,d,kl,q],1))
    u_x = tf.gradients(u,x)
    return u,u_x

@tf.function
def grad(model, x_f, t_f, d_f, kl_f, q_f,
         x0, t0, d0, kl0, q0,
         x_lb, t_lb, d_lb, kl_lb, q_lb,
         x_ub, t_ub, d_ub, kl_ub, q_ub,
         ub1_weights, ub2_weights, ui_weights):
    
    with tf.GradientTape(persistent=True) as tape:
        #tape.watch(col_weights)
        #tape.watch(u_weights)
        loss_value, mse_0, mse_f, mse_b_u = loss(x_f, t_f, d_f, kl_f, q_f,
                                                                 x0, t0, d0, kl0, q0,
                                                                 x_lb, t_lb, d_lb, kl_lb, q_lb,
                                                                 x_ub, t_ub, d_ub, kl_ub, q_ub,
                                                                 ub1_weights, ub2_weights, ui_weights)
        
        grads = tape.gradient(loss_value, u_model.trainable_variables)
        #print(grads)
#         grads_colf = tape.gradient(loss_value, colf_weights)
        grads_ub1 = tape.gradient(loss_value, ub1_weights)
        grads_ub2 = tape.gradient(loss_value, ub2_weights)
        grads_ui = tape.gradient(loss_value, ui_weights)
        
        
#         grads_u = tape.gradient(loss_value, u_weights)
#         grads_colfx = tape.gradient(loss_value, colfx_weights)
#         grads_colft = tape.gradient(loss_value, colft_weights)
#         

    return loss_value, mse_0, mse_f, mse_b_u, grads, grads_ub1, grads_ub2, grads_ui

def fit(x_f, t_f, d_f, kl_f, q_f, x0, t0, d0, kl0, q0, x_lb, t_lb, d_lb, kl_lb, q_lb, x_ub, t_ub, d_ub, kl_ub, q_ub, ub1_weights, ub2_weights, ui_weights, tf_iter, newton_iter):
    # Built in support for mini-batch, set to N_f (i.e. full batch) by default
    batch_sz = 100000
    n_batches =  N_f // batch_sz
    start_time = time.time()
#     global_step = tf.Variable(0, trainable = False)
    learning_rate1 = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=2e-3, decay_steps=5000, decay_rate=.9, staircase=False)
    learning_rate2 = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=4e-3, decay_steps=4000, decay_rate=.9, staircase=False)
    
    
#     tf_optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate1, weight_decay=0.004, amsgrad=False)
#     tf_optimizer_colf = tfa.optimizers.AdamW(learning_rate=learning_rate2, weight_decay=0.003, amsgrad=False)
#     tf_optimizer_colfx = tfa.optimizers.AdamW(learning_rate=learning_rate2, weight_decay=0.003, amsgrad=False)
#     tf_optimizer_colft = tfa.optimizers.AdamW(learning_rate=learning_rate2, weight_decay=0.003, amsgrad=False)
#     tf_optimizer_ub1 = tfa.optimizers.AdamW(learning_rate=learning_rate2, weight_decay=0.003, amsgrad=False)
#     tf_optimizer_ub2 = tfa.optimizers.AdamW(learning_rate=learning_rate2, weight_decay=0.003, amsgrad=False)
#     tf_optimizer_ui = tfa.optimizers.AdamW(learning_rate=learning_rate2, weight_decay=0.003, amsgrad=False)
    
    tf_optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate1, beta_1=.90)
#     tf_optimizer2 = tf.keras.optimizers.Adam(learning_rate = learning_rate2, beta_1=.90)
#     tf_optimizer_u = tf.keras.optimizers.Adam(learning_rate = learning_rate2, beta_1=.90)
#     tf_optimizer_colf = tf.keras.optimizers.Adam(learning_rate = learning_rate2, beta_1=.90)
#     tf_optimizer_colfx = tf.keras.optimizers.Adam(learning_rate = learning_rate2, beta_1=.90)
#     tf_optimizer_colft = tf.keras.optimizers.Adam(learning_rate = learning_rate2, beta_1=.90)
    tf_optimizer_ub1 = tf.keras.optimizers.Adam(learning_rate = learning_rate2, beta_1=.90)
    tf_optimizer_ub2 = tf.keras.optimizers.Adam(learning_rate = learning_rate2, beta_1=.90)
    tf_optimizer_ui = tf.keras.optimizers.Adam(learning_rate = learning_rate2, beta_1=.90)
    
    print("starting Adam training")

    for epoch in range(tf_iter):
        for i in range(n_batches):

            x_f_batch = x_f[i*batch_sz:(i*batch_sz + batch_sz),]
            t_f_batch = t_f[i*batch_sz:(i*batch_sz + batch_sz),]
            d_f_batch = d_f[i*batch_sz:(i*batch_sz + batch_sz),]
            kl_f_batch = kl_f[i*batch_sz:(i*batch_sz + batch_sz),]
            q_f_batch = q_f[i*batch_sz:(i*batch_sz + batch_sz),]

            loss_value, mse_0, mse_f, mse_b_u, grads, grads_ub1, grads_ub2, grads_ui = grad(u_model, 
                                                                    x_f_batch, t_f_batch, d_f_batch, kl_f_batch, q_f_batch,
                                                                    x0, t0, d0, kl0, q0, x_lb, t_lb, d_lb, kl_lb, q_lb,
                                                                    x_ub, t_ub, d_ub, kl_ub, q_ub,
                                                                    ub1_weights, ub2_weights, ui_weights)
            
            tf_optimizer.apply_gradients(zip(grads, u_model.trainable_variables))
#             tf_optimizer_u.apply_gradients(zip([-grads_u], [u_weights]))
#             tf_optimizer_colf.apply_gradients(zip([-grads_colf], [colf_weights]))
#             tf_optimizer_colfx.apply_gradients(zip([-grads_colfx], [colfx_weights]))
#             tf_optimizer_colft.apply_gradients(zip([-grads_colft], [colft_weights]))
            tf_optimizer_ub1.apply_gradients(zip([-grads_ub1], [ub1_weights]))
            tf_optimizer_ub2.apply_gradients(zip([-grads_ub2], [ub2_weights]))
            tf_optimizer_ui.apply_gradients(zip([-grads_ui], [ui_weights]))


        if epoch % 100 == 0:
#             print(grads)
            elapsed = time.time() - start_time
            print('It: %d, Time: %.2f' % (epoch, elapsed))
            tf.print(f"mse_0: {mse_0}  mse_f: {mse_f} mse_b_u: {mse_b_u} total loss: {loss_value}")
            start_time = time.time()


    #l-bfgs-b optimization
#     print("Starting L-BFGS training")

#     loss_and_flat_grad = get_loss_and_flat_grad(x_f, t_f, d_f, kl_f, q_f, x0, t0, d0, kl0, q0, x_lb, t_lb, d_lb, kl_lb, q_lb, x_ub, t_ub, d_ub, kl_ub, q_ub, ub1_weights, ub2_weights, ui_weights)

#     lbfgs(loss_and_flat_grad, get_weights(u_model), Struct(), maxIter=newton_iter, learningRate=.01)


# L-BFGS implementation from https://github.com/pierremtb/PINNs-TF2.0
def get_loss_and_flat_grad(x_f, t_f, d_f, kl_f, q_f, x0, t0, d0, kl0, q0, x_lb, t_lb, d_lb, kl_lb, q_lb, x_ub, t_ub, d_ub, kl_ub, q_ub, ub1_weights, ub2_weights, ui_weights):
    def loss_and_flat_grad(w):
        with tf.GradientTape() as tape:
            set_weights(u_model, w, sizes_w, sizes_b)
            loss_value, _, _, _ = loss(x_f, t_f, d_f, kl_f, q_f, x0, t0, d0, kl0, q0, x_lb, t_lb, d_lb, kl_lb, q_lb, 
                                       x_ub, t_ub, d_ub, kl_ub, q_ub, ub1_weights, ub2_weights, ui_weights)
        grad = tape.gradient(loss_value, u_model.trainable_variables)
        grad_flat = []
        for g in grad:
            grad_flat.append(tf.reshape(g, [-1]))
        grad_flat = tf.concat(grad_flat, 0)
        #print(loss_value, grad_flat)
        return loss_value, grad_flat

    return loss_and_flat_grad


def predict(X_star):
    X_star = tf.convert_to_tensor(X_star, dtype=tf.float32)
    u_star, _ = u_x_model(X_star[:,0:1], X_star[:,1:2], X_star[:,2:3], X_star[:,3:4], X_star[:,4:5])

#     f_u_star, f_gx_star, f_gt_star = f_model(X_star[:,0:1], X_star[:,1:2], X_star[:,2:3])

    return np.array(u_star)



### please uncomment the stages correspondingly, wherever you see Stage i for i = 0 to 4###

#Stage 0
N0 = 1000
N_b = 1000
N_f = 100000

# #Stage 1
# N0 = 5000
# N_b = 5000
# N_f = 200000

# #Stage 2
# N0 = 25000
# N_b = 50000
# N_f = 1000000

# #Stage 3
# N0 = 100000
# N_b = 150000
# N_f = 3000000


ub1_weights1 = tf.Variable(tf.reshape(tf.repeat(1.0, N_b),(N_b, -1)))
ub2_weights1 = tf.Variable(tf.reshape(tf.repeat(1.0, N_b),(N_b, -1)))
ui_weights1 = tf.Variable(tf.reshape(tf.repeat(1.0, N0),(N0, -1)))

colf_weights = tf.Variable(tf.reshape(tf.repeat(1.0, N_f),(N_f, -1)))
colfx_weights = tf.Variable(tf.reshape(tf.repeat(1.0, N_f),(N_f, -1)))
colft_weights = tf.Variable(tf.reshape(tf.repeat(1.0, N_f),(N_f, -1)))
u_weights = tf.Variable(tf.reshape(tf.repeat(1.0, N_u),(N_u, -1)))

t = tf.convert_to_tensor(np.linspace(0,15,100)[:,None], dtype=tf.float32)

x = tf.convert_to_tensor(np.linspace(0,30,100)[:,None], dtype=tf.float32)

#Stage 0
d = 15.
kl = .05
q = 5.0

# #Stage 2
# d = tf.convert_to_tensor(np.linspace(3,30,50)[:,None], dtype=tf.float32)
# kl = .05
# q = 5.0

# #Stage 3
# d = tf.convert_to_tensor(np.linspace(3,30,50)[:,None], dtype=tf.float32)
# kl = tf.convert_to_tensor(np.linspace(.01,.1,50)[:,None], dtype=tf.float32)
# q = 5.0

# #Stage 4
# d = tf.convert_to_tensor(np.linspace(3,30,50)[:,None], dtype=tf.float32)
# kl = tf.convert_to_tensor(np.linspace(.01,.1,50)[:,None], dtype=tf.float32)
# q = tf.convert_to_tensor(np.linspace(1,10,50)[:,None], dtype=tf.float32)


X_init, T_init, D_init, KL_init, Q_init  = np.meshgrid(x, 0, d, kl, q)
X_bc1, T_bc1, D_bc1, KL_bc1, Q_bc1  = np.meshgrid(0, t, d, kl, q)
X_bc2, T_bc2, D_bc2, KL_bc2, Q_bc2  = np.meshgrid(30, t, d, kl, q)   

xxi = np.hstack((X_init.flatten()[:,None], T_init.flatten()[:,None], D_init.flatten()[:,None], KL_init.flatten()[:,None], Q_init.flatten()[:,None]))
idx_init = np.random.choice(xxi.shape[0], N0, replace=False)
xxi = xxi[idx_init, :]
xxbc1 = np.hstack((X_bc1.flatten()[:,None], T_bc1.flatten()[:,None], D_bc1.flatten()[:,None], KL_bc1.flatten()[:,None], Q_bc1.flatten()[:,None]))
idx_bc1 = np.random.choice(xxbc1.shape[0], N_b, replace=False)
xxbc1 = xxbc1[idx_bc1, :]
xxbc2 = np.hstack((X_bc2.flatten()[:,None], T_bc2.flatten()[:,None], D_bc2.flatten()[:,None], KL_bc2.flatten()[:,None], Q_bc2.flatten()[:,None]))
idx_bc2 = np.random.choice(xxbc2.shape[0], N_b, replace=False)
xxbc2 = xxbc2[idx_bc2, :]


#Stage 0
XF = tf.math.sobol_sample(2, N_f, dtype=tf.float32)
XF_x = 30*XF[:,0:1]
XF_t = 15*XF[:,1:2]
# XF_d = 3 + (30 - 3)*XF[:,2:3]
# XF_kl = .01 + (.1 - .01)*XF[:,3:4]
# XF_q = 1 + (10 - 1)*XF[:,4:5]

XF_d = tf.reshape(tf.repeat(10., N_f),(N_f, -1))
XF_kl = tf.reshape(tf.repeat(.05, N_f),(N_f, -1))
XF_q = tf.reshape(tf.repeat(5.0, N_f),(N_f, -1))

# #Stage 1
# XF = tf.math.sobol_sample(3, N_f, dtype=tf.float32)
# XF_x = 30*XF[:,0:1]
# XF_t = 15*XF[:,1:2]
# XF_d = 3 + (30 - 3)*XF[:,2:3]
# # XF_kl = .01 + (.1 - .01)*XF[:,3:4]
# # XF_q = 1 + (10 - 1)*XF[:,4:5]

# XF_kl = tf.reshape(tf.repeat(.05, N_f),(N_f, -1))
# XF_q = tf.reshape(tf.repeat(5.0, N_f),(N_f, -1))

# #Stage 2
# XF = tf.math.sobol_sample(4, N_f, dtype=tf.float32)
# XF_x = 30*XF[:,0:1]
# XF_t = 15*XF[:,1:2]
# XF_d = 3 + (30 - 3)*XF[:,2:3]
# XF_kl = .01 + (.1 - .01)*XF[:,3:4]
# # XF_q = 1 + (10 - 1)*XF[:,4:5]

# XF_q = tf.reshape(tf.repeat(5.0, N_f),(N_f, -1))

# #Stage 4
# XF = tf.math.sobol_sample(5, N_f, dtype=tf.float32)
# XF_x = 30*XF[:,0:1]
# XF_t = 15*XF[:,1:2]
# XF_d = 3 + (30 - 3)*XF[:,2:3]
# XF_kl = .01 + (.1 - .01)*XF[:,3:4]
# XF_q = 1 + (10 - 1)*XF[:,4:5]

XFF = tf.squeeze(tf.stack([XF_x, XF_t, XF_d, XF_kl, XF_q], axis=1))


x0 = tf.convert_to_tensor(xxi[:,0:1], dtype=tf.float32)
t0 = tf.convert_to_tensor(xxi[:,1:2], dtype=tf.float32)
d0 = tf.convert_to_tensor(xxi[:,2:3], dtype=tf.float32)
kl0 = tf.convert_to_tensor(xxi[:,3:4], dtype=tf.float32)
q0 = tf.convert_to_tensor(xxi[:,4:5], dtype=tf.float32)

x_lb = tf.convert_to_tensor(xxbc1[:,0:1], dtype=tf.float32)
t_lb = tf.convert_to_tensor(xxbc1[:,1:2], dtype=tf.float32)
d_lb = tf.convert_to_tensor(xxbc1[:,2:3], dtype=tf.float32)
kl_lb = tf.convert_to_tensor(xxbc1[:,3:4], dtype=tf.float32)
q_lb = tf.convert_to_tensor(xxbc1[:,4:5], dtype=tf.float32)

x_ub = tf.convert_to_tensor(xxbc2[:,0:1], dtype=tf.float32)
t_ub = tf.convert_to_tensor(xxbc2[:,1:2], dtype=tf.float32)
d_ub = tf.convert_to_tensor(xxbc2[:,2:3], dtype=tf.float32)
kl_ub = tf.convert_to_tensor(xxbc2[:,3:4], dtype=tf.float32)
q_ub = tf.convert_to_tensor(xxbc2[:,4:5], dtype=tf.float32)


# data = scipy.io.loadmat('USAPINNsAnalytic.mat')
# tt = data['t'].flatten()[:,None][:]
# xx = data['x'].flatten()[:,None]
# dd = np.asarray([1, 11, 101])[:,None]
# # dd = data['d'].flatten()[:,None][:21]

# X_u, T_u, D_u  = np.meshgrid(xx, tt, dd)
# U_Exact1 = data['sols'][:,:,0]
# U_Exact2 = data['sols'][:,:,2]
# U_Exact3 = data['sols'][:,:,20]
# dataa = np.stack((U_Exact1, U_Exact2, U_Exact3), axis=2)
# U_Exact = np.real(np.transpose(dataa, (1, 0, 2))).flatten()[:,None]

# xxu = np.hstack((X_u.flatten()[:,None], T_u.flatten()[:,None], D_u.flatten()[:,None]))
# idx_u = np.random.choice(xxu.shape[0], N_u, replace=False)
# xxu = xxu[idx_u, :]
# U_Exact = U_Exact[idx_u, :]

# Xs_x = tf.convert_to_tensor(xxu[:,0:1], dtype=tf.float32)
# Xs_t = tf.convert_to_tensor(xxu[:,1:2], dtype=tf.float32)
# Xs_d = tf.convert_to_tensor(xxu[:,2:3], dtype=tf.float32)
# U_star = tf.convert_to_tensor(U_Exact, dtype=tf.float32)


# %matplotlib inline
# fig = plt.figure()
# plt.rcParams["figure.figsize"] = (8.4,6.4)
# ax = fig.add_subplot(projection='3d')

# # ax.scatter(xxi[:,2], xxi[:,3], xxi[:,4], color = 'k', alpha=.3, s=.1, label = '$init$')
# # ax.scatter(xxbc1[:,2], xxbc1[:,3], xxbc1[:,4], color = 'b', alpha=.3, s=.1, label = '$bc1$')
# # ax.scatter(xxbc2[:,2], xxbc2[:,3], xxbc2[:,4], color = "c", alpha=.3, s=.1, label = '$bc2$')
# ax.scatter(XFF[:,2], XFF[:,3], XFF[:,4], color = "r", alpha=.25, s=.1, label = '$f$')
# # ax.scatter(xxu[:,1], xxu[:,0], xxu[:,2], color = "k", marker='*', alpha=.55, s=10, label = '$u$')

# ax.set_xlabel('$d$', fontsize= 30)
# ax.set_ylabel('$kl$', fontsize= 30)
# ax.set_zlabel('$q$')
# ax.set_title('$ X^{Dis} $' , fontsize= 30)
# plt.legend(loc='upper right', markerscale=8)
# plt.show()



NameError: name 'Sequential' is not defined

In [ ]:
loss_and_flat_grad = get_loss_and_flat_grad(XF_x, XF_t, XF_d, XF_kl, XF_q, x0, t0, d0, kl0, q0, x_lb, t_lb, d_lb, kl_lb, q_lb, x_ub, t_ub, d_ub, kl_ub, q_ub, ub1_weights1, ub2_weights1, ui_weights1)
lbfgs(loss_and_flat_grad, get_weights(u_model), Struct(), maxIter=1000, learningRate=.8)

In [ ]:
%matplotlib inline
fig = plt.figure()
plt.rcParams["figure.figsize"] = (8.4,6.4)
ax = fig.add_subplot(projection='3d')

# ax.scatter(xxi[:,1], xxi[:,3], xxi[:,4], color = 'k', alpha=.5, s=.2, label = '$init$')
ax.scatter(xxbc1[:,0], xxbc1[:,1], xxbc1[:,2], color = 'b', alpha=.5, s=.2, label = '$bc1$')
ax.scatter(xxbc2[:,0], xxbc2[:,3], xxbc2[:,4], color = "c", alpha=.5, s=.2, label = '$bc2$')
# ax.scatter(XFF[:,1], XFF[:,3], XFF[:,4], color = "r", alpha=.5, s=.2, label = '$f$')
# ax.scatter(xxu[:,1], xxu[:,0], xxu[:,2], color = "k", marker='*', alpha=.55, s=10, label = '$u$')

ax.set_xlabel('$d$', fontsize= 30)
ax.set_ylabel('$kl$', fontsize= 30)
ax.set_zlabel('$q$')
ax.set_title('$ X^{Dis} $' , fontsize= 30)
plt.legend(loc='upper right', markerscale=8)
plt.show()

In [3]:
fit(XF_x, XF_t, XF_d, XF_kl, XF_q, x0, t0, d0, kl0, q0, x_lb, t_lb, d_lb, kl_lb, q_lb, x_ub, t_ub, d_ub, kl_ub, q_ub, ub1_weights1, ub2_weights1, ui_weights1, tf_iter = 150000, newton_iter = 10000)

starting Adam training
It: 0, Time: 24.14
mse_0: 9.964995384216309  mse_f: 0.6146932244300842 mse_b_u: 1235.8365478515625 total loss: 1246.416259765625
It: 100, Time: 29.25
mse_0: 0.1378343403339386  mse_f: 0.04551170393824577 mse_b_u: 0.013792603276669979 total loss: 0.19713865220546722
It: 200, Time: 29.24
mse_0: 0.0851173922419548  mse_f: 0.0439925491809845 mse_b_u: 0.013777628540992737 total loss: 0.14288757741451263
It: 300, Time: 29.20
mse_0: 0.06048007309436798  mse_f: 0.041617460548877716 mse_b_u: 0.0111074298620224 total loss: 0.1132049635052681
It: 400, Time: 29.19
mse_0: 0.04831897094845772  mse_f: 0.04079239070415497 mse_b_u: 0.010071980766952038 total loss: 0.0991833359003067
It: 500, Time: 29.18
mse_0: 0.04075348749756813  mse_f: 0.039742715656757355 mse_b_u: 0.00883621908724308 total loss: 0.08933242410421371
It: 600, Time: 29.17
mse_0: 0.03578364476561546  mse_f: 0.038289982825517654 mse_b_u: 0.00790861714631319 total loss: 0.08198224753141403
It: 700, Time: 29.19
mse_0

It: 5600, Time: 29.20
mse_0: 0.03028237260878086  mse_f: 0.022702451795339584 mse_b_u: 0.001560149365104735 total loss: 0.054544974118471146
It: 5700, Time: 29.19
mse_0: 0.02654034085571766  mse_f: 0.020373079925775528 mse_b_u: 0.030147800222039223 total loss: 0.07706122100353241
It: 5800, Time: 29.20
mse_0: 0.030526671558618546  mse_f: 0.018027033656835556 mse_b_u: 0.05131648853421211 total loss: 0.09987019002437592
It: 5900, Time: 29.18
mse_0: 0.01804007776081562  mse_f: 0.018199758604168892 mse_b_u: 0.00172336061950773 total loss: 0.03796319663524628
It: 6000, Time: 29.19
mse_0: 0.02464759163558483  mse_f: 0.013569836504757404 mse_b_u: 0.006697630975395441 total loss: 0.04491506144404411
It: 6100, Time: 29.19
mse_0: 0.01609119400382042  mse_f: 0.01443201582878828 mse_b_u: 0.02002873830497265 total loss: 0.05055195093154907
It: 6200, Time: 29.18
mse_0: 0.013321178033947945  mse_f: 0.01453999150544405 mse_b_u: 0.002789183519780636 total loss: 0.03065035492181778
It: 6300, Time: 29.19


It: 11400, Time: 29.19
mse_0: 0.002768212230876088  mse_f: 0.0014451290480792522 mse_b_u: 0.00844113714993 total loss: 0.012654478661715984
It: 11500, Time: 29.21
mse_0: 0.0041830213740468025  mse_f: 0.001320448936894536 mse_b_u: 0.011657415889203548 total loss: 0.01716088503599167
It: 11600, Time: 29.19
mse_0: 0.003151778830215335  mse_f: 0.001464769127778709 mse_b_u: 0.016735926270484924 total loss: 0.02135247364640236
It: 11700, Time: 29.21
mse_0: 0.03894256427884102  mse_f: 0.010871144942939281 mse_b_u: 0.007682875730097294 total loss: 0.057496584951877594
It: 11800, Time: 29.21
mse_0: 0.003453805111348629  mse_f: 0.0024235378950834274 mse_b_u: 0.005479455925524235 total loss: 0.011356798931956291
It: 11900, Time: 29.19
mse_0: 0.002632948337122798  mse_f: 0.0016025189543142915 mse_b_u: 0.003439896972849965 total loss: 0.00767536461353302
It: 12000, Time: 29.19
mse_0: 0.0032644213642925024  mse_f: 0.0014588084304705262 mse_b_u: 0.009512223303318024 total loss: 0.014235453680157661
I

It: 17100, Time: 29.21
mse_0: 0.002813664497807622  mse_f: 0.0007412118720822036 mse_b_u: 0.033958714455366135 total loss: 0.03751359134912491
It: 17200, Time: 29.23
mse_0: 0.0016010270919650793  mse_f: 0.0007937804912216961 mse_b_u: 0.020786592736840248 total loss: 0.023181401193141937
It: 17300, Time: 29.17
mse_0: 0.004028006922453642  mse_f: 0.000676409516017884 mse_b_u: 0.04476174712181091 total loss: 0.04946616291999817
It: 17400, Time: 29.19
mse_0: 0.00135109294205904  mse_f: 0.0008847378776408732 mse_b_u: 0.004597598221153021 total loss: 0.006833428982645273
It: 17500, Time: 29.20
mse_0: 0.0024417832028120756  mse_f: 0.000750998966395855 mse_b_u: 0.0034221753012388945 total loss: 0.006614957470446825
It: 17600, Time: 29.20
mse_0: 0.001148480107076466  mse_f: 0.0008513492066413164 mse_b_u: 0.01506707351654768 total loss: 0.01706690341234207
It: 17700, Time: 29.19
mse_0: 0.0012550720712170005  mse_f: 0.0008060714462772012 mse_b_u: 0.010266606695950031 total loss: 0.012327750213444

It: 22800, Time: 29.21
mse_0: 0.0009576506563462317  mse_f: 0.0005868162261322141 mse_b_u: 0.01759098842740059 total loss: 0.019135454669594765
It: 22900, Time: 29.20
mse_0: 0.001434648991562426  mse_f: 0.000533927814103663 mse_b_u: 0.017273662611842155 total loss: 0.019242240116000175
It: 23000, Time: 29.19
mse_0: 0.0011615571565926075  mse_f: 0.0005424041883088648 mse_b_u: 0.001052377512678504 total loss: 0.0027563387993723154
It: 23100, Time: 29.18
mse_0: 0.0006048218929208815  mse_f: 0.0005985474563203752 mse_b_u: 0.00019453270942904055 total loss: 0.0013979020295664668
It: 23200, Time: 29.21
mse_0: 0.0008682119660079479  mse_f: 0.0005496357916854322 mse_b_u: 0.0016547504346817732 total loss: 0.003072598250582814
It: 23300, Time: 29.18
mse_0: 0.001258759992197156  mse_f: 0.0005002490943297744 mse_b_u: 0.0073442780412733555 total loss: 0.009103287011384964
It: 23400, Time: 29.18
mse_0: 0.0007792740943841636  mse_f: 0.0005680242902599275 mse_b_u: 0.003830556757748127 total loss: 0.00

...

It: 123700, Time: 29.17
mse_0: 9.24153755477164e-06  mse_f: 6.514639972010627e-05 mse_b_u: 0.00032852403819561005 total loss: 0.00040291197365149856
It: 123800, Time: 29.19
mse_0: 1.0014558029070031e-05  mse_f: 6.475362897617742e-05 mse_b_u: 0.0003570161061361432 total loss: 0.00043178428313694894
It: 123900, Time: 29.20
mse_0: 9.840246093517635e-06  mse_f: 6.469986692536622e-05 mse_b_u: 0.0003865367325488478 total loss: 0.00046107685193419456
It: 124000, Time: 29.18
mse_0: 9.679255526862107e-06  mse_f: 6.513974949484691e-05 mse_b_u: 0.00024320797820109874 total loss: 0.00031802698504179716
It: 124100, Time: 29.20
mse_0: 1.0123870197276119e-05  mse_f: 6.447156920330599e-05 mse_b_u: 0.0004230585473123938 total loss: 0.0004976539639756083
It: 124200, Time: 29.19
mse_0: 9.399965165357571e-06  mse_f: 6.467229104600847e-05 mse_b_u: 0.0004494382010307163 total loss: 0.0005235104472376406
It: 124300, Time: 29.22
mse_0: 9.293408766097855e-06  mse_f: 6.491402018582448e-05 mse_b_u: 0.00024181891

It: 129200, Time: 29.18
mse_0: 9.615750968805514e-06  mse_f: 6.348292663460597e-05 mse_b_u: 0.0003002754820045084 total loss: 0.00037337414687499404
It: 129300, Time: 29.17
mse_0: 9.345995749754366e-06  mse_f: 6.38794299447909e-05 mse_b_u: 0.0002852005127351731 total loss: 0.0003584259538911283
It: 129400, Time: 29.18
mse_0: 1.1371969776519109e-05  mse_f: 6.414467497961596e-05 mse_b_u: 0.0003784226137213409 total loss: 0.00045393925393000245
It: 129500, Time: 29.20
mse_0: 1.0678668331820518e-05  mse_f: 6.403592124115676e-05 mse_b_u: 0.00035396567545831203 total loss: 0.0004286802723072469
It: 129600, Time: 29.20
mse_0: 9.707365279609803e-06  mse_f: 6.306414434220642e-05 mse_b_u: 0.00029235752299427986 total loss: 0.0003651290317066014
It: 129700, Time: 29.19
mse_0: 9.27094242797466e-06  mse_f: 6.3521925767418e-05 mse_b_u: 0.00040492360130883753 total loss: 0.0004777164722327143
It: 129800, Time: 29.18
mse_0: 1.0384645975136664e-05  mse_f: 6.30550566711463e-05 mse_b_u: 0.000327566725900

It: 134700, Time: 29.17
mse_0: 1.2036876796628349e-05  mse_f: 6.242822564672679e-05 mse_b_u: 0.0002550304925534874 total loss: 0.00032949558226391673
It: 134800, Time: 29.19
mse_0: 8.775304195296485e-06  mse_f: 6.281198147917166e-05 mse_b_u: 0.0003031218075193465 total loss: 0.0003747090813703835
It: 134900, Time: 29.20
mse_0: 9.340393262391444e-06  mse_f: 6.22714142082259e-05 mse_b_u: 0.00035027958801947534 total loss: 0.0004218913963995874
It: 135000, Time: 29.18
mse_0: 8.995233656605706e-06  mse_f: 6.2232677009888e-05 mse_b_u: 0.00029154380899854004 total loss: 0.0003627717378549278
It: 135100, Time: 29.19
mse_0: 8.64020603330573e-06  mse_f: 6.238490459509194e-05 mse_b_u: 0.00029139971593394876 total loss: 0.0003624248201958835
It: 135200, Time: 29.18
mse_0: 9.535525350656826e-06  mse_f: 6.240788206923753e-05 mse_b_u: 0.00040123690268956125 total loss: 0.0004731803201138973
It: 135300, Time: 29.20
mse_0: 8.708123459655326e-06  mse_f: 6.231373117771e-05 mse_b_u: 0.0002402632671874016

It: 140300, Time: 29.18
mse_0: 8.618283573014196e-06  mse_f: 6.159812619443983e-05 mse_b_u: 0.0002619884617161006 total loss: 0.00033220485784113407
It: 140400, Time: 29.18
mse_0: 8.264921234513167e-06  mse_f: 6.157940515549853e-05 mse_b_u: 0.000248430558713153 total loss: 0.0003182748914696276
It: 140500, Time: 29.20
mse_0: 8.49304433359066e-06  mse_f: 6.163620128063485e-05 mse_b_u: 0.0003165115194860846 total loss: 0.0003866407787427306
It: 140600, Time: 29.22
mse_0: 9.505184607405681e-06  mse_f: 6.153651338536292e-05 mse_b_u: 0.0002543488226365298 total loss: 0.0003253905160818249
It: 140700, Time: 29.19
mse_0: 8.894350685295649e-06  mse_f: 6.146737723611295e-05 mse_b_u: 0.00030049867928028107 total loss: 0.00037086039083078504
It: 140800, Time: 29.18
mse_0: 8.413457180722617e-06  mse_f: 6.157071038614959e-05 mse_b_u: 0.00023912964388728142 total loss: 0.00030911382054910064
It: 140900, Time: 29.18
mse_0: 8.691784387337975e-06  mse_f: 6.152811693027616e-05 mse_b_u: 0.000178178612259

It: 145900, Time: 29.20
mse_0: 1.034617525874637e-05  mse_f: 6.049630610505119e-05 mse_b_u: 0.00031470946851186454 total loss: 0.0003855519462376833
It: 146000, Time: 29.19
mse_0: 8.456346222374123e-06  mse_f: 6.045068585081026e-05 mse_b_u: 0.0002438045630697161 total loss: 0.00031271157786250114
It: 146100, Time: 29.17
mse_0: 8.1253538155579e-06  mse_f: 6.0682272305712104e-05 mse_b_u: 0.0002873401390388608 total loss: 0.0003561477642506361
It: 146200, Time: 29.18
mse_0: 7.956088666105643e-06  mse_f: 6.104473868617788e-05 mse_b_u: 0.0002920770784839988 total loss: 0.0003610778949223459
It: 146300, Time: 29.20
mse_0: 8.151871043082792e-06  mse_f: 6.0835092881461605e-05 mse_b_u: 0.00034899156889878213 total loss: 0.00041797853191383183
It: 146400, Time: 29.19
mse_0: 7.968288628035225e-06  mse_f: 6.0670463426504284e-05 mse_b_u: 0.00021916962577961385 total loss: 0.00028780836146324873
It: 146500, Time: 29.19
mse_0: 8.1829111877596e-06  mse_f: 6.056545316823758e-05 mse_b_u: 0.0002881586551

In [4]:
np.save('ub1_weights1', ub1_weights1)
np.save('ub2_weights1', ub2_weights1)
np.save('ui_weights1', ui_weights1)
np.save('X', x)
np.save('T', t)
np.save('D', d)
np.save('KL', kl)
np.save('Q', q)
u_model.save("u_model")

INFO:tensorflow:Assets written to: u_model\assets


In [ ]:
np.max(ub1_weights2)

In [ ]:
N0 = 2000
N_b = 4000
N_f = 10000

t = np.linspace(0,15,100)[:,None]
t = tf.convert_to_tensor(t, dtype=tf.float32)

x = np.linspace(0,30,100)[:,None]
x = tf.convert_to_tensor(x, dtype=tf.float32)

d = np.linspace(1,300,100)[:,None]
d = tf.convert_to_tensor(d, dtype=tf.float32)

kl = np.linspace(.01,1,100)[:,None]
kl = tf.convert_to_tensor(kl, dtype=tf.float32)

q = np.linspace(.1,10,100)[:,None]
q = tf.convert_to_tensor(q, dtype=tf.float32)


X_init, T_init, D_init, KL_init, Q_init  = np.meshgrid(x, 0, d, kl, q)
X_bc1, T_bc1, D_bc1, KL_bc1, Q_bc1  = np.meshgrid(0, t, d, kl, q)
X_bc2, T_bc2, D_bc2, KL_bc2, Q_bc2  = np.meshgrid(30, t, d, kl, q)   

xxi = np.hstack((X_init.flatten()[:,None], T_init.flatten()[:,None], D_init.flatten()[:,None], KL_init.flatten()[:,None], Q_init.flatten()[:,None]))
idx_init = np.random.choice(xxi.shape[0], N0, replace=False)
xxi = xxi[idx_init, :]
xxbc1 = np.hstack((X_bc1.flatten()[:,None], T_bc1.flatten()[:,None], D_bc1.flatten()[:,None], KL_bc1.flatten()[:,None], Q_bc1.flatten()[:,None]))
idx_bc1 = np.random.choice(xxbc1.shape[0], N_b, replace=False)
xxbc1 = xxbc1[idx_bc1, :]
xxbc2 = np.hstack((X_bc2.flatten()[:,None], T_bc2.flatten()[:,None], D_bc2.flatten()[:,None], KL_bc2.flatten()[:,None], Q_bc2.flatten()[:,None]))
idx_bc2 = np.random.choice(xxbc2.shape[0], N_b, replace=False)
xxbc2 = xxbc2[idx_bc2, :]



XF = tf.math.sobol_sample(5, N_f, dtype=tf.float32)
XF_x = 30*XF[:,0:1]
XF_t = 15*XF[:,1:2]
XF_d = 1 + (300 - 1)*XF[:,2:3]
XF_kl = .01 + (1 - .01)*XF[:,3:4]
XF_q = .1 + (10 - .1)*XF[:,4:5]

# XF_kl = tf.reshape(tf.repeat(.1, N_f),(N_f, -1))
# XF_q = tf.reshape(tf.repeat(.5, N_f),(N_f, -1))

XFF = tf.squeeze(tf.stack([XF_x, XF_t, XF_d, XF_kl, XF_q], axis=1))

In [ ]:
%matplotlib inline
fig = plt.figure()
plt.rcParams["figure.figsize"] = (8.4,6.4)
ax = fig.add_subplot()

ax.scatter(xxi[:,1], xxi[:,0], color = 'k', alpha=.5, s=.7, label = '$init$')
ax.scatter(xxbc1[:,1], xxbc1[:,0], color = 'b', alpha=.5, s=.5, label = '$bc1$')
ax.scatter(xxbc2[:,1], xxbc2[:,0], color = "c", alpha=.5, s=.5, label = '$bc2$')
ax.scatter(XFF[:,1], XFF[:,0], color = "r", alpha=.1, s=.5, label = '$f$')

ax.scatter(xxi[:,1]+15, xxi[:,0]+30, color = 'k', alpha=.5, s=.7, label = '$init$')
ax.scatter(xxbc1[:,1]+15, xxbc1[:,0]+30, color = 'b', alpha=.5, s=.5, label = '$bc1$')
ax.scatter(xxbc2[:,1]+15, xxbc2[:,0]+30, color = "c", alpha=.5, s=.5, label = '$bc2$')
ax.scatter(XFF[:,1]+15, XFF[:,0]+30, color = "r", alpha=.1, s=.5, label = '$f$')

ax.scatter(xxi[:,1]+45, xxi[:,0]+90, color = 'k', alpha=.5, s=.7, label = '$init$')
ax.scatter(xxbc1[:,1]+45, xxbc1[:,0]+90, color = 'b', alpha=.5, s=.5, label = '$bc1$')
ax.scatter(xxbc2[:,1]+45, xxbc2[:,0]+90, color = "c", alpha=.5, s=.5, label = '$bc2$')
ax.scatter(XFF[:,1]+45, XFF[:,0]+90, color = "r", alpha=.1, s=.5, label = '$f$')
# ax.scatter(xxu[:,1], xxu[:,0], xxu[:,2], color = "k", marker='*', alpha=.55, s=10, label = '$u$')

ax.set_xlabel('$t$', fontsize= 20)
ax.set_ylabel('$x$', fontsize= 20)
# ax.set_zlabel('$d$', fontsize= 20)
ax.set_title('$ X^{Distribution} $' , fontsize= 20)
# plt.legend(loc='upper right', markerscale=6)
plt.show()

In [ ]:
# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=(22.4, 24.8))
ax = fig.add_subplot(3, 3, 1, projection='3d')
sc = ax.scatter(XFF[:,1], XFF[:,0], XFF[:,2], label = '$f$', c = np.asarray(colf_weights), s = np.asarray(colf_weights)/5)
plt.colorbar(sc, aspect=10, shrink = .5)
# ax = fig.add_subplot(3, 3, 2, projection='3d')
# sc1 = ax.scatter(XFF[:,1], XFF[:,0], XFF[:,2], label = '$fgx$', c = np.asarray(colfx_weights), s = np.asarray(colfx_weights)/5)
# plt.colorbar(sc1, aspect=10, shrink = .5)
# ax = fig.add_subplot(3, 3, 3, projection='3d')
# sc2 = ax.scatter(XFF[:,1], XFF[:,0], XFF[:,2], label = '$fgt$', c = np.asarray(colft_weights), s = np.asarray(colft_weights)/5)
# plt.colorbar(sc2, aspect=10, shrink = .5)
# ax = fig.add_subplot(3, 3, 4, projection='3d')
# sc3 = ax.scatter(xxu[:,1], xxu[:,0], xxu[:,2], label = '$u$', c = np.asarray(u_weights), s = np.asarray(u_weights)/5)
# plt.colorbar(sc3, aspect=10, shrink = .5)
# ax = fig.add_subplot(3, 3, 5, projection='3d')
# sc4 = ax.scatter(xxi[:,1], xxi[:,0], xxi[:,2], label = '$i$', c = np.asarray(ui_weights), s = np.asarray(ui_weights)/5)
# plt.colorbar(sc4, aspect=10, shrink = .5)
# ax = fig.add_subplot(3, 3, 6, projection='3d')
# sc5 = ax.scatter(xxbc1[:,1], xxbc1[:,0], xxbc1[:,2], label = '$bc1$', c = np.asarray(ub1_weights), s = np.asarray(ub1_weights)/5)
# plt.colorbar(sc5, aspect=10, shrink = .5)
# ax = fig.add_subplot(3, 3, 8, projection='3d')
# sc6 = ax.scatter(xxbc2[:,1], xxbc2[:,0], xxbc2[:,2], label = '$bc2$', c = np.asarray(ub2_weights), s = np.asarray(ub2_weights)/5)
# plt.colorbar(sc6, aspect=10, shrink = .5)
plt.show()

In [ ]:
u_model = keras.models.load_model('u_model')

In [ ]:
u_model = keras.models.load_model('ExUiPinns results/Model9-3 250-mish/Stage 2/u_model')

In [ ]:
# model = keras.models.load_model('USAgPINNs-TF2-GPU-results/Test10 - w obs - tanh - w dropout - pec 1.5 - pec 300/u_model')

@tf.function
def f_model(x, t, d):
    #pe = tf.constant(3, dtype=tf.float32)
    u = model(tf.concat([x,t,d], 1))
    u_x = tf.gradients(u,x)[0]
    #u_xx = tf.gradients(u_x, x)
    u_t = tf.gradients(u,t)[0]
    ff = 10*u - tf.math.multiply(d,u_x)
    ff_x = tf.gradients(ff, x)[0]
    f = u_t + ff_x
    f_t = tf.gradients(f,t)[0]
    f_x = tf.gradients(f,x)[0]

    return f, f_x, f_t

@tf.function
def u_x_model(x, t, d):
    u = model(tf.concat([x,t,d],1))
    u_x = tf.gradients(u,x)
    return u,u_x

def predict(X_star):
    X_star = tf.convert_to_tensor(X_star, dtype=tf.float32)
    u_star, _ = u_x_model(X_star[:,0:1], X_star[:,1:2], X_star[:,2:3])

    f_u_star, f_gx_star, f_gt_star = f_model(X_star[:,0:1], X_star[:,1:2], X_star[:,2:3])

    return np.array(u_star), np.array(f_u_star), np.squeeze(np.array(f_gx_star))[:,None], np.squeeze(np.array(f_gt_star))[:,None]


t = 15*tf.math.sobol_sample(1, 400)
x = 30*tf.math.sobol_sample(1, 500)
d = 1 + 199*tf.math.sobol_sample(1, 200)

In [ ]:
kl1 = np.linspace(.01,.1,100)
q1 = np.linspace(10,1,100)
1 + (1.587/.37)*((kl1*q1)/(1+kl1)**2)
z = 1 + (1.587/.37)*((kl1*q1)/(1+kl1)**2)

In [ ]:
plt.plot(kl1 ,z)
plt.xscale('log',base=10) 
# plt.yscale('log',base=10) 

In [ ]:
plt.plot(q1,z)
plt.xscale('log',base=10) 

In [ ]:
kl1 = .02
q1 = 2
1 + (1.587/.37)*((kl1*q1)/(1+kl1)**2)

In [ ]:
X, T, D, KL, Q  = np.meshgrid(x, t, 10, .02, 2)
X_star  = np.hstack((X.flatten()[:,None], T.flatten()[:,None], D.flatten()[:,None], KL.flatten()[:,None], Q.flatten()[:,None]))
idx = np.random.choice(X_star.shape[0], 10000, replace=False)
X_star = tf.convert_to_tensor(X_star[idx, :], dtype=tf.float32)
#u_pred, f_pred = model.predict(X_star)
u_pred = predict(X_star)

#error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
#print('Error u: %e' % (error_u))                     


U_pred = griddata(X_star[:,0:2], u_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')
# f_pred = griddata(X_star[:,0:2], f_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')
# fgx_pred = griddata(X_star[:,0:2], fgx_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')
# fgt_pred = griddata(X_star[:,0:2], fgt_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')

In [ ]:
fig = plt.figure()
gridspec.GridSpec(1,1)

ax = plt.subplot2grid((1,1), (0,0))

levels = np.linspace(-0.05, 1.05, 500)

CS_pred1 = ax.contourf(np.squeeze(T), np.squeeze(X), U_pred, levels=levels, cmap='jet', origin='lower')


# CS_pred1 = ax.contourf(T1, X1, U1_pred, cmap='jet', origin='lower')
# CS_pred2 = ax.contourf(T2, X2, U2_pred, cmap='jet', origin='lower')
# CS_pred3 = ax.contourf(T3, X3, U3_pred, cmap='jet', origin='lower')

cbar = fig.colorbar(CS_pred1)
cbar.ax.tick_params(labelsize=20)
ax.set_xlim(-0.13, 15.13)
ax.set_ylim(-0.51, 30.51)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_title('$ C^{prediction}_{Pec=3}$', fontsize= 30)

ax.set_xlabel('$T$')
ax.set_ylabel('$X$')
ax.set_title('$f USAPINN D 11$', fontsize= 30)


fig.set_size_inches(w=10,h=8)
#fig.savefig('CPINNs_Uncertain_Pec.png')

In [ ]:
X, T, D, KL, Q  = np.meshgrid(30, t, 10, kl, 1)
X_star  = np.hstack((X.flatten()[:,None], T.flatten()[:,None], D.flatten()[:,None], KL.flatten()[:,None], Q.flatten()[:,None]))
idx = np.random.choice(X_star.shape[0], 10000, replace=False)
X_star = tf.convert_to_tensor(X_star[idx, :], dtype=tf.float32)
#u_pred, f_pred = model.predict(X_star)
u_pred = predict(X_star)

#error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
#print('Error u: %e' % (error_u))                     


U_pred = griddata((X_star[:,1],X_star[:,3]), u_pred.flatten(), (np.squeeze(T), np.squeeze(KL)), method='cubic')
# f_pred = griddata(X_star[:,0:2], f_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')
# fgx_pred = griddata(X_star[:,0:2], fgx_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')
# fgt_pred = griddata(X_star[:,0:2], fgt_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')

In [ ]:
fig = plt.figure()
gridspec.GridSpec(1,1)

ax = plt.subplot2grid((1,1), (0,0))

levels = np.linspace(-0.05, 1.05, 500)

CS_pred1 = ax.contourf(np.squeeze(T), np.squeeze(KL), U_pred, levels=levels, cmap='jet', origin='lower')


# CS_pred1 = ax.contourf(T1, X1, U1_pred, cmap='jet', origin='lower')
# CS_pred2 = ax.contourf(T2, X2, U2_pred, cmap='jet', origin='lower')
# CS_pred3 = ax.contourf(T3, X3, U3_pred, cmap='jet', origin='lower')

cbar = fig.colorbar(CS_pred1)
cbar.ax.tick_params(labelsize=20)
ax.set_xlim(-0.13, 15.13)
ax.set_ylim(0, 1.01)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_title('$ C^{prediction}_{Pec=3}$', fontsize= 30)

ax.set_xlabel('$T$', fontsize= 20)
ax.set_ylabel('$KL$', fontsize= 20)
ax.set_title('Concentration Variability at Effluent - Peclet = 30', fontsize= 17)


fig.set_size_inches(w=10,h=8)
#fig.savefig('CPINNs_Uncertain_Pec.png')

In [ ]:
X, T, D, KL, Q  = np.meshgrid(30, 15, d, kl, 3)
X_star  = np.hstack((X.flatten()[:,None], T.flatten()[:,None], D.flatten()[:,None], KL.flatten()[:,None], Q.flatten()[:,None]))
idx = np.random.choice(X_star.shape[0], 10000, replace=False)
X_star = tf.convert_to_tensor(X_star[idx, :], dtype=tf.float32)
#u_pred, f_pred = model.predict(X_star)
u_pred = predict(X_star)

#error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
#print('Error u: %e' % (error_u))                     


U_pred = griddata(X_star[:,2:4], u_pred.flatten(), (np.squeeze(D), np.squeeze(KL)), method='cubic')
# f_pred = griddata(X_star[:,0:2], f_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')
# fgx_pred = griddata(X_star[:,0:2], fgx_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')
# fgt_pred = griddata(X_star[:,0:2], fgt_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')

In [ ]:
fig = plt.figure()
gridspec.GridSpec(1,1)

ax = plt.subplot2grid((1,1), (0,0))

levels = np.linspace(-0.05, 1.05, 500)

CS_pred1 = ax.contourf(np.squeeze(D), np.squeeze(KL), U_pred, levels=levels, cmap='jet', origin='lower')


# CS_pred1 = ax.contourf(T1, X1, U1_pred, cmap='jet', origin='lower')
# CS_pred2 = ax.contourf(T2, X2, U2_pred, cmap='jet', origin='lower')
# CS_pred3 = ax.contourf(T3, X3, U3_pred, cmap='jet', origin='lower')

cbar = fig.colorbar(CS_pred1)
cbar.ax.tick_params(labelsize=20)
ax.set_xlim(-1, 303)
ax.set_ylim(0, 1.01)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_title('$ C^{prediction}_{Pec=3}$', fontsize= 30)

ax.set_xlabel('$D$', fontsize= 20)
ax.set_ylabel('$KL$', fontsize= 20)
ax.set_title('Concentration Variability at Effluent - End time', fontsize= 15)


fig.set_size_inches(w=10,h=8)
#fig.savefig('CPINNs_Uncertain_Pec.png')

In [ ]:
X, T, D, KL, Q  = np.meshgrid(30, 15, 100, kl, q)
X_star  = np.hstack((X.flatten()[:,None], T.flatten()[:,None], D.flatten()[:,None], KL.flatten()[:,None], Q.flatten()[:,None]))
idx = np.random.choice(X_star.shape[0], 10000, replace=False)
X_star = tf.convert_to_tensor(X_star[idx, :], dtype=tf.float32)
#u_pred, f_pred = model.predict(X_star)
u_pred = predict(X_star)

#error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
#print('Error u: %e' % (error_u))                     


U_pred = griddata(X_star[:,3:5], u_pred.flatten(), (np.squeeze(KL), np.squeeze(Q)), method='cubic')
# f_pred = griddata(X_star[:,0:2], f_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')
# fgx_pred = griddata(X_star[:,0:2], fgx_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')
# fgt_pred = griddata(X_star[:,0:2], fgt_pred.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')

In [ ]:
fig = plt.figure()
gridspec.GridSpec(1,1)

ax = plt.subplot2grid((1,1), (0,0))

levels = np.linspace(-0.05, 1.05, 500)

CS_pred1 = ax.contourf(np.squeeze(KL), np.squeeze(Q), U_pred, levels=levels, cmap='jet', origin='lower')


# CS_pred1 = ax.contourf(T1, X1, U1_pred, cmap='jet', origin='lower')
# CS_pred2 = ax.contourf(T2, X2, U2_pred, cmap='jet', origin='lower')
# CS_pred3 = ax.contourf(T3, X3, U3_pred, cmap='jet', origin='lower')

cbar = fig.colorbar(CS_pred1)
cbar.ax.tick_params(labelsize=20)
ax.set_xlim(0, 1.01)
ax.set_ylim(0, 10.1)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_title('$ C^{prediction}_{Pec=3}$', fontsize= 30)

ax.set_xlabel('$KL$', fontsize= 20)
ax.set_ylabel('$Q$', fontsize= 20)
ax.set_title('Concentration Variability at Effluent - End time - Peclet = 3', fontsize= 15)


fig.set_size_inches(w=10,h=8)
#fig.savefig('CPINNs_Uncertain_Pec.png')

In [ ]:
fig = plt.figure()
gridspec.GridSpec(1,1)

ax = plt.subplot2grid((1,1), (0,0))

levels = np.linspace(-0.04, .08, 500)

CS_pred1 = ax.contourf(np.squeeze(T), np.squeeze(X), f_pred, levels=levels, cmap='jet', origin='lower')


# CS_pred1 = ax.contourf(T1, X1, U1_pred, cmap='jet', origin='lower')
# CS_pred2 = ax.contourf(T2, X2, U2_pred, cmap='jet', origin='lower')
# CS_pred3 = ax.contourf(T3, X3, U3_pred, cmap='jet', origin='lower')

cbar = fig.colorbar(CS_pred1)
cbar.ax.tick_params(labelsize=20)
ax.set_xlim(-0.13, 15.13)
ax.set_ylim(-0.51, 30.51)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_title('$ C^{prediction}_{Pec=3}$', fontsize= 30)

ax.set_xlabel('$T$')
ax.set_ylabel('$X$')
ax.set_title('$f USAPINN D 11$', fontsize= 30)


fig.set_size_inches(w=10,h=8)
#fig.savefig('CPINNs_Uncertain_Pec.png')

In [ ]:
fig = plt.figure()
gridspec.GridSpec(1,1)

ax = plt.subplot2grid((1,1), (0,0))

levels = np.linspace(-.02, .04, 500)

CS_pred1 = ax.contourf(np.squeeze(T), np.squeeze(X), fgx_pred, levels=levels, cmap='jet', origin='lower')


# CS_pred1 = ax.contourf(T1, X1, U1_pred, cmap='jet', origin='lower')
# CS_pred2 = ax.contourf(T2, X2, U2_pred, cmap='jet', origin='lower')
# CS_pred3 = ax.contourf(T3, X3, U3_pred, cmap='jet', origin='lower')

cbar = fig.colorbar(CS_pred1)
cbar.ax.tick_params(labelsize=20)
ax.set_xlim(-0.13, 15.13)
ax.set_ylim(-0.51, 30.51)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_title('$ C^{prediction}_{Pec=3}$', fontsize= 30)

ax.set_xlabel('$T$')
ax.set_ylabel('$X$')
ax.set_title('$fgx USAPINN D 11$', fontsize= 30)


fig.set_size_inches(w=10,h=8)
#fig.savefig('CPINNs_Uncertain_Pec.png')

In [ ]:
fig = plt.figure()
gridspec.GridSpec(1,1)

ax = plt.subplot2grid((1,1), (0,0))

levels = np.linspace(-.2, .4, 500)

CS_pred1 = ax.contourf(np.squeeze(T), np.squeeze(X), fgt_pred, levels=levels, cmap='jet', origin='lower')


# CS_pred1 = ax.contourf(T1, X1, U1_pred, cmap='jet', origin='lower')
# CS_pred2 = ax.contourf(T2, X2, U2_pred, cmap='jet', origin='lower')
# CS_pred3 = ax.contourf(T3, X3, U3_pred, cmap='jet', origin='lower')

cbar = fig.colorbar(CS_pred1)
cbar.ax.tick_params(labelsize=20)
ax.set_xlim(-0.13, 15.13)
ax.set_ylim(-0.51, 30.51)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_title('$ C^{prediction}_{Pec=3}$', fontsize= 30)

ax.set_xlabel('$T$')
ax.set_ylabel('$X$')
ax.set_title('$fgt USAPINN D 11$', fontsize= 30)


fig.set_size_inches(w=10,h=8)
#fig.savefig('CPINNs_Uncertain_Pec.png')

In [ ]:
data = scipy.io.loadmat('USAPINNsAnalytic.mat')
tt = data['t'].flatten()[:,None]
xx = data['x'].flatten()[:,None]
dd = data['d'].flatten()[:,None]
Exact = np.real(data['sols'])[:,:,0].T

XX, TT= np.meshgrid(xx, tt)
XX_star  = np.hstack((XX.flatten()[:,None], TT.flatten()[:,None]))
UU_pred = griddata(XX_star, Exact.flatten(), (np.squeeze(X), np.squeeze(T)), method='cubic')

In [ ]:
data['d'].flatten()[:,None][0]

In [ ]:
300/201

In [ ]:
dd = data['d'].flatten()[:,None]
dd

In [ ]:
fig = plt.figure()
gridspec.GridSpec(1,1)

ax = plt.subplot2grid((1,1), (0,0))

levels = np.linspace(-0.05, 1.05, 500)

CS_pred1 = ax.contourf(np.squeeze(T), np.squeeze(X), UU_pred, levels=levels, cmap='jet', origin='lower')


# CS_pred1 = ax.contourf(T1, X1, U1_pred, cmap='jet', origin='lower')
# CS_pred2 = ax.contourf(T2, X2, U2_pred, cmap='jet', origin='lower')
# CS_pred3 = ax.contourf(T3, X3, U3_pred, cmap='jet', origin='lower')

cbar = fig.colorbar(CS_pred1)
cbar.ax.tick_params(labelsize=20)
ax.set_xlim(-0.13, 15.13)
ax.set_ylim(-0.51, 30.51)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_title('$ C^{prediction}_{Pec=3}$', fontsize= 30)

ax.set_xlabel('$T$')
ax.set_ylabel('$X$')
ax.set_title('$Analytical D 11$', fontsize= 30)


fig.set_size_inches(w=10,h=8)
#fig.savefig('CPINNs_Uncertain_Pec.png')

In [ ]:
fig = plt.figure()
gridspec.GridSpec(1,1)

ax = plt.subplot2grid((1,1), (0,0))

levels = np.linspace(0, .1, 500)

CS_pred1 = ax.contourf(np.squeeze(T), np.squeeze(X), np.abs(U_pred - UU_pred), levels=levels, cmap='jet', origin='lower')


# CS_pred1 = ax.contourf(T1, X1, U1_pred, cmap='jet', origin='lower')
# CS_pred2 = ax.contourf(T2, X2, U2_pred, cmap='jet', origin='lower')
# CS_pred3 = ax.contourf(T3, X3, U3_pred, cmap='jet', origin='lower')

cbar = fig.colorbar(CS_pred1)
cbar.ax.tick_params(labelsize=20)
ax.set_xlim(-0.13, 15.13)
ax.set_ylim(-0.51, 30.51)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_title('$ C^{prediction}_{Pec=3}$', fontsize= 30)

ax.set_xlabel('$T$')
ax.set_ylabel('$X$')
ax.set_title('$Error D 11$', fontsize= 30)


fig.set_size_inches(w=10,h=8)
#fig.savefig('CPINNs_Uncertain_Pec.png')

In [ ]:
#find L2 error
U_pred = np.nan_to_num(U_pred, nan=0)
error_u = np.linalg.norm(UU_pred-U_pred,2)/np.linalg.norm(UU_pred,2)
print('Error u: %e' % (error_u))

In [ ]:
# Plot the results
#Column Loc
j = -1
fig, ax = plt.subplots()
ax.set_xlabel('Time (hr)', size = 12, weight = 'bold')
ax.set_ylabel('Concentration (mg/L)', size = 12, weight = 'bold')
ax.set_title('Column Breakthrough Curve $Pe = 300$', size = 14, weight = 'bold')
# ax.plot(np.squeeze(T[:-1,j]), UU_pred[:-1,j], ls = '-', c = 'r', label = 'Breakthrough curve FDM')
ax.plot(np.squeeze(T[:-1,j]), U_pred[:-1,j], ls = '-', c = 'k', label = 'Breakthrough curve PINNS')

# Add a couple of other lines for explanation of behavior
xs = [0, 30/10, 30/10, 15]
ys = [0, 0, 1, 1]
ax.plot(xs, ys, ls = '-.', lw = 1, c = 'b', label = 'Plug flow')
ax.text(0.7,.68,'Effects of Dispersion', fontsize = 12)
arrowprops = {'arrowstyle':'<|-|>'}
ax.annotate('', xy = (5.5,.65), xytext = (.5,.65), ha = 'right', va = 'center', arrowprops = arrowprops)
leg = ax.legend()
plt.savefig('breakthrough')